In [162]:
import pandas as pd
import numpy as np
import mido

In [163]:
mid = mido.MidiFile("../Data/Fugue-BWV-847.mid")

In [164]:
midiframe = pd.DataFrame(columns=["Voice", "Pitch", "Velocity", "Ticks"])

In [172]:
for track in mid.tracks:
    for msg in track:
        if msg.type == 'note_on':
            df = pd.DataFrame({"Voice":msg.channel, "Pitch":msg.note, "Velocity":msg.velocity, "Ticks":msg.time},\
                              index=[0])
            midiframe = midiframe.append(df, ignore_index=True)

midiframe.loc[0,'Ticks'] = 0

In [176]:
midiframe['Time'] = pd.Series(midiframe['Ticks'].cumsum()/480, index=midiframe.index)

In [177]:
note_on = pd.DataFrame(midiframe[midiframe['Velocity'] != 0])
note_off = pd.DataFrame(midiframe[midiframe['Velocity'] == 0])
newdex = range(0,len(note_off))
newdex

range(0, 1502)

In [178]:
note_off = note_off.reset_index(drop=True)
note_on = note_on.reset_index(drop=True)

In [181]:
note_on['Duration'] = pd.Series(note_off['Time'] - note_on['Time'], index = note_on.index)
#NOTE: TIME IS STILL MEASURED IN TICKS!!!

In [182]:
note_on = note_on

,Pitch,Ticks,Time,Velocity,Voice,Duration
0,79,0,0.000000,64,0,0.235417
1,78,7,0.250000,64,0,0.235417
2,79,7,0.500000,64,0,0.472917
3,72,13,1.000000,64,0,0.472917
4,75,13,1.500000,64,0,0.472917
5,79,13,2.000000,64,0,0.235417
6,78,7,2.250000,64,0,0.235417
7,79,7,2.500000,64,0,0.472917
8,81,13,3.000000,64,0,0.472917
9,74,13,3.500000,64,0,0.472917


In [173]:
mid.ticks_per_beat

480